In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report , recall_score ,  precision_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.exceptions import DataConversionWarning
from sklearn.metrics import accuracy_score

In [55]:
flight = pd.read_csv(r"C:\LHL\Week_2\Dey_5\flight_delay_predictions\ammar\flights_sorted.csv")
flight.head(3)

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime
0,12/31/2019,AA,4815,11057,CLT,"Charlotte, NC",10208,AGS,"Augusta, GA",1125,1117.0,-8.0,27.0,1144.0,1214.0,5.0,1227,1219.0,-8.0,0,0,62,62.0,30.0,1,140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12/31/2019,DL,2239,14747,SEA,"Seattle, WA",14107,PHX,"Phoenix, AZ",1859,1858.0,-1.0,23.0,1921.0,2236.0,6.0,2244,2242.0,-2.0,0,0,165,164.0,135.0,1,1107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12/31/2019,UA,461,11292,DEN,"Denver, CO",12264,IAD,"Washington, DC",1340,1334.0,-6.0,14.0,1348.0,1823.0,6.0,1848,1829.0,-19.0,0,0,188,175.0,155.0,1,1452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
pd.set_option('display.max_columns', None)


In [57]:
variables_to_remove = [  'mkt_carrier_fl_num', 'mkt_carrier_fl_num', 'origin_airport_id',
                         'dest_airport_id', 'wheels_off', 'wheels_on']

In [58]:
flight.drop(variables_to_remove, axis = 1, inplace = True)

In [59]:
missing_df = flight.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(flight.shape[0]-missing_df['missing values'])/flight.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)


,variable,missing values,filling factor (%)
0,longest_add_gtime,99356,0.644
1,first_dep_time,99356,0.644
2,total_add_gtime,99356,0.644
3,late_aircraft_delay,81041,18.959
4,security_delay,81041,18.959
5,nas_delay,81041,18.959
6,weather_delay,81041,18.959
7,carrier_delay,81041,18.959
8,air_time,1960,98.040
9,arr_delay,1935,98.065


In [60]:
flight['late_aircraft_delay'] = flight['late_aircraft_delay'].fillna(0)
flight['security_delay'] = flight['security_delay'].fillna(0)
flight['nas_delay'] = flight['nas_delay'].fillna(0)
flight['carrier_delay'] = flight['carrier_delay'].fillna(0)
flight['weather_delay'] = flight['weather_delay'].fillna(0)

In [61]:
missing_df = flight.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(flight.shape[0]-missing_df['missing values'])/flight.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,longest_add_gtime,99356,0.644
1,first_dep_time,99356,0.644
2,total_add_gtime,99356,0.644
3,air_time,1960,98.040
4,arr_delay,1935,98.065
5,actual_elapsed_time,1922,98.078
6,taxi_in,1751,98.249
7,arr_time,1713,98.287
8,taxi_out,1705,98.295
9,dep_delay,1652,98.348


In [62]:
flight.drop(['longest_add_gtime','first_dep_time','total_add_gtime'], axis = 1, inplace = True)

In [63]:
flight['pre_flight_delay'] = flight['late_aircraft_delay'] + flight['security_delay'] + flight['nas_delay'] + flight['weather_delay'] + flight['carrier_delay']

In [64]:
flight['late_aircraft_delay'] = flight['late_aircraft_delay'].fillna(0)
flight['security_delay'] = flight['security_delay'].fillna(0)
flight['nas_delay'] = flight['nas_delay'].fillna(0)
flight['carrier_delay'] = flight['carrier_delay'].fillna(0)
flight['weather_delay'] = flight['weather_delay'].fillna(0)

In [65]:
flight.dropna(inplace=True)
flight.head(5)

,fl_date,mkt_unique_carrier,origin,origin_city_name,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,pre_flight_delay
0,12/31/2019,AA,CLT,"Charlotte, NC",AGS,"Augusta, GA",1125,1117.0,-8.0,27.0,5.0,1227,1219.0,-8.0,0,0,62,62.0,30.0,1,140,0.0,0.0,0.0,0.0,0.0,0.0
1,12/31/2019,DL,SEA,"Seattle, WA",PHX,"Phoenix, AZ",1859,1858.0,-1.0,23.0,6.0,2244,2242.0,-2.0,0,0,165,164.0,135.0,1,1107,0.0,0.0,0.0,0.0,0.0,0.0
2,12/31/2019,UA,DEN,"Denver, CO",IAD,"Washington, DC",1340,1334.0,-6.0,14.0,6.0,1848,1829.0,-19.0,0,0,188,175.0,155.0,1,1452,0.0,0.0,0.0,0.0,0.0,0.0
3,12/31/2019,WN,SFO,"San Francisco, CA",BUR,"Burbank, CA",1600,1624.0,24.0,6.0,6.0,1715,1727.0,12.0,0,0,75,63.0,51.0,1,326,0.0,0.0,0.0,0.0,0.0,0.0
4,12/31/2019,UA,EWR,"Newark, NJ",CHS,"Charleston, SC",1000,957.0,-3.0,22.0,7.0,1228,1209.0,-19.0,0,0,148,132.0,103.0,1,628,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
flight['fl_date'] = pd.to_datetime(flight['fl_date'], format= "%m/%d/%Y" )

In [67]:
flight['month'] = pd.to_datetime(flight['fl_date']).dt.month

In [68]:
flight['day'] = pd.to_datetime(flight['fl_date']).dt.day

In [69]:
flight.head(3)

,fl_date,mkt_unique_carrier,origin,origin_city_name,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,pre_flight_delay,month,day
0,2019-12-31,AA,CLT,"Charlotte, NC",AGS,"Augusta, GA",1125,1117.0,-8.0,27.0,5.0,1227,1219.0,-8.0,0,0,62,62.0,30.0,1,140,0.0,0.0,0.0,0.0,0.0,0.0,12,31
1,2019-12-31,DL,SEA,"Seattle, WA",PHX,"Phoenix, AZ",1859,1858.0,-1.0,23.0,6.0,2244,2242.0,-2.0,0,0,165,164.0,135.0,1,1107,0.0,0.0,0.0,0.0,0.0,0.0,12,31
2,2019-12-31,UA,DEN,"Denver, CO",IAD,"Washington, DC",1340,1334.0,-6.0,14.0,6.0,1848,1829.0,-19.0,0,0,188,175.0,155.0,1,1452,0.0,0.0,0.0,0.0,0.0,0.0,12,31


In [70]:
flight['dep_time'] = flight['dep_time'].astype(str).str.strip()

In [71]:
#convert dep_time to H:M
flight['dep_time'] = flight['dep_time'].map(str)
#flight.dep_time = flight.dep_time.str[:2] + ':' + flight.dep_time.str[-2:]
flight['dep_time'] = flight['dep_time'].astype(str).replace('\.', '', regex=True)
flight['dep_time'] = flight['dep_time'].str.zfill(4)



In [72]:
#convert crs_arr_time to H:M
flight['crs_arr_time'] = flight['crs_arr_time'].map(str)
#flight.crs_arr_time = flight.crs_arr_time.str[:2] + ':' + flight.crs_arr_time.str[-2:]
flight['crs_arr_time'] = flight['crs_arr_time'].str.zfill(4)

In [73]:
#convert arr_time to H:M
#flight.arr_time = flight.arr_time.str[:2] + ':' + flight.arr_time .str[-2:]
flight['arr_time'] = flight['arr_time'].map(str)
flight['dep_time'] = flight['dep_time'].astype(str).replace('\.', '', regex=True)
flight['arr_time'] = flight['arr_time'].str.zfill(4)

In [74]:
#convert crs_dep_time to H:M
flight['crs_dep_time'] = flight['crs_dep_time'].map(str)
#flight.crs_dep_time = flight.crs_dep_time.str[:2] + ':' + flight.arr_time .str[-2:]
flight['crs_dep_time'] = flight['crs_dep_time'].astype(str).replace('\.', '', regex=True)
flight['crs_dep_time'] = flight['crs_dep_time'].str.zfill(4)

#flight['crs_dep_time'] = pd.to_datetime(flight['crs_dep_time']).dt.strftime('%H:%M')

In [75]:
flight.head(2)

,fl_date,mkt_unique_carrier,origin,origin_city_name,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,diverted,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,pre_flight_delay,month,day
0,2019-12-31,AA,CLT,"Charlotte, NC",AGS,"Augusta, GA",1125,11170,-8.0,27.0,5.0,1227,1219.0,-8.0,0,0,62,62.0,30.0,1,140,0.0,0.0,0.0,0.0,0.0,0.0,12,31
1,2019-12-31,DL,SEA,"Seattle, WA",PHX,"Phoenix, AZ",1859,18580,-1.0,23.0,6.0,2244,2242.0,-2.0,0,0,165,164.0,135.0,1,1107,0.0,0.0,0.0,0.0,0.0,0.0,12,31


In [76]:
flight['crs_dep_time'].unique

<bound method Series.unique of 0        1125
1        1859
2        1340
3        1600
4        1000
         ... 
99995    2048
99996    1425
99997    0820
99998    0600
99999    2016
Name: crs_dep_time, Length: 97994, dtype: object>

In [77]:
# Convert time to 24 hours
flight['24_hours'] = flight.crs_dep_time.str[:2]

### Convert categorical data to numerical

In [78]:
def encode_categories(features):
    lb_make = LabelEncoder()
    for i in range(len(features)):
        flight[features[i]] = lb_make.fit_transform(flight[features[i]])

encode_categories(['mkt_unique_carrier','origin','dest','dep_time'])

### Removing original encoded columns

In [79]:
flight.drop(['origin_city_name','dest_city_name', 'fl_date',
 'arr_delay', 'taxi_out', 'taxi_in', 'taxi_in', 'diverted',
  'carrier_delay','weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 
  'cancelled','flights',  ], axis=1,  inplace = True)

### Splitting our Data Set into Training Data and Test Data


In [80]:
flight = flight.rename(columns={'dep_delay':'TARGET'})

In [81]:
feature_names = ['day','month','mkt_unique_carrier','origin','dest','dep_time','distance']
X =  flight[feature_names].values
y =  flight['TARGET'].values

In [83]:
def get_accuracy(y_true, y_preds):
    # Getting score of confusion matrix
    true_negative, false_positive, false_negative, true_positive = confusion_matrix(y_true, y_preds).ravel()
    # Calculating accuracy
    accuracy = (true_positive + true_negative)/(true_negative + false_positive + false_negative + true_positive)
    return accuracy

In [84]:
# Splitting Train-set and Test-set
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=41)

# Splitting Train-set and Validation-set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=41)

In [85]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

Logistic Regression



In [86]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0).fit(X_train, y_train)

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Random Forest Classifier

In [87]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# Make predictions for the test set
y_pred_test = rf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred_test)

0.08178988723914485

KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=2)

Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

GridSearchCV

In [ ]:
from datetime import datetime
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [ ]:
startTimeGS = datetime.now()
from sklearn.model_selection import GridSearchCV
param_grid = {
                 'n_estimators': [10],
#                 'min_samples_split': [2, 4],
#                 'min_samples_leaf': [2, 4],
#                 'max_features': ['sqrt', 'log2'],
                 "criterion" : ["gini"]
             }
grid_rf = GridSearchCV(rf, param_grid, cv=3)
grid_rf.fit(X_train, y_train)
bestModel = grid_rf.best_estimator_
bestParameters = grid_rf.best_params_
gridScores = grid_rf.param_grid
print('Random forest Grid Search with non-redundant variables took [', datetime.now() - startTimeGS, '] seconds.')

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


Random forest Grid Search with non-redundant variables took [ 0:00:45.016066 ] seconds.


In [ ]:
print(bestModel)
print(bestParameters)
gridScores

RandomForestClassifier(n_estimators=10)
{'criterion': 'gini', 'n_estimators': 10}


{'n_estimators': [10], 'criterion': ['gini']}

Evaluation of accuracy on validation dataset

In [ ]:
models = [lr, gnb, rf, knn]
acc = []
for model in models:
    preds_val = model.predict(X_val)
    accuracy = get_accuracy(y_val, preds_val)
    acc.append(accuracy)

ValueError: too many values to unpack (expected 4)

In [ ]:
model_name = ['Logistic Regression', 'Catboost', 'Naive Bayes', 'Random Forest', 'KNN']
accuracy = dict(zip(model_name, acc))

In [ ]:
plt.figure(figsize=(15,5))
ax = sns.barplot(x = list(accuracy.keys()), y = list(accuracy.values()))
for p, value in zip(ax.patches, list(accuracy.values())):
    _x = p.get_x() + p.get_width() / 2
    _y = p.get_y() + p.get_height() + 0.008
    ax.text(_x, _y, round(value, 3), ha="center") 
plt.xlabel("Models")
plt.ylabel("Accuracy")
plt.title("Model vs. Accuracy")
plt.show()